## Imports

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
import requests
from typing import Annotated
import json

In [3]:
# create tool
@tool
def multiply(a: int, b: int)->int:
    """multiplies two numbers"""
    return a*b

@tool
def addition(a: int, b: int)->int:
    """adds two numbers"""
    return a+b

In [4]:
# instantiate llm
llm = ChatOpenAI(model = "gpt-4o-mini")

# bind to llm
llm_with_tools = llm.bind_tools([multiply,addition])

In [6]:
print(llm_with_tools.invoke("What is the capital of India?"))

content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 73, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b83a7d52ea', 'id': 'chatcmpl-CEz6JHWS5BdCRRDxoro0PZVew64qu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ed3a7725-d88a-440d-aa77-e834978002fc-0' usage_metadata={'input_tokens': 73, 'output_tokens': 9, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
llm_with_tools.invoke("What is 10 times 50?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_g5aHbLkpxl0k9CobJq9GwJ3B', 'function': {'arguments': '{"a":10,"b":50}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 74, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CFDsNaTJFz87L0vRayVuxKqnMga0G', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--43848029-89b4-4cb7-a3c1-4876df143fd1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 50}, 'id': 'call_g5aHbLkpxl0k9CobJq9GwJ3B', 'type': 'tool_call'}], usage_metadata={'input_tokens': 74, 'output_tokens': 17, 'total_tokens': 91, 'input_token_details': {'audio': 0, '

In [6]:
llm_with_tools.invoke("What is 50 increased by 50?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i5nDTrOaOOnBTevlbFGUNTbv', 'function': {'arguments': '{"a":50,"b":50}', 'name': 'addition'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 75, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CFDtXkkK4bMSRbsBIuedRaQuLiz20', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5204bd7f-29fb-474a-8198-9a43f9861c5c-0', tool_calls=[{'name': 'addition', 'args': {'a': 50, 'b': 50}, 'id': 'call_i5nDTrOaOOnBTevlbFGUNTbv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 75, 'output_tokens': 18, 'total_tokens': 93, 'input_token_details': {'audio': 0, '

## Tool execution

In [7]:
result = llm_with_tools.invoke("What is 50 increased by 50?")

In [8]:
result.tool_calls

[{'name': 'addition',
  'args': {'a': 50, 'b': 50},
  'id': 'call_OzupreIaNMzp4rxEjlT1N0Lu',
  'type': 'tool_call'}]

In [11]:
addition.invoke(result.tool_calls[0])

ToolMessage(content='100', name='addition', tool_call_id='call_OzupreIaNMzp4rxEjlT1N0Lu')

## Currency Conversion Tool

In [37]:
# conversion factor api tool
@tool
def fetch_conversion_factor(base_currency: str, target_currency: str)-> float:
    """fetches the latest conversion factor from base currency to target currency"""
    base_currency = base_currency.upper()
    target_currency = target_currency.upper()
    
    url = f"https://v6.exchangerate-api.com/v6/aaf05154f5eeafdf92f45a0a/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()

# multiplying with conevrsion factor tool
@tool
def convert(base_value: float, conversion_factor: Annotated[float, InjectedToolArg])->float:
    """given a currency rate this function calculates the target currency value from given base currency value"""
    return round(base_value*conversion_factor,2)

In [29]:
fetch_conversion_factor.invoke({'base_currency': 'aed', 'target_currency': 'inr'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757721601,
 'time_last_update_utc': 'Sat, 13 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1757808001,
 'time_next_update_utc': 'Sun, 14 Sep 2025 00:00:01 +0000',
 'base_code': 'AED',
 'target_code': 'INR',
 'conversion_rate': 24.0483}

In [30]:
convert.invoke({"base_value":10,"conversion_factor":24})

240.0

In [38]:
# tool binding
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([convert,fetch_conversion_factor])

In [39]:
query = "What is the conversion rate between usd and inr, and based on that can you convert 10 usd to inr?"

In [40]:
result = llm_with_tools.invoke(query)

In [41]:
result.tool_calls

[{'name': 'fetch_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_xtuR9mVFAL9zUjdbGxJYiAA9',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_value': 10},
  'id': 'call_67M4Ax0Z3tWXr0qH2UatFwhS',
  'type': 'tool_call'}]

In [45]:
for tool_call in result.tool_calls:
    # execute first tool to get conversion factor
    if tool_call['name'] == 'fetch_conversion_factor':
        tool_message_1 = fetch_conversion_factor.invoke(tool_call)
        # print(tool_message_1)
        conversion_rate = json.loads(tool_message_1.content)['conversion_rate']
        print(conversion_rate)
        
    
    # execute second tool using conversion rate from tool 1

88.3176
